In [1]:
import sys,os,json
import collections,math
import time,datetime,pytz
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from collections import Counter
from matplotlib.colors import ListedColormap
import seaborn as sns
import networkx as nx
# from textblob import TextBlob
# from polyglot.detect import Detector
import operator
# import PlotUtil
from scipy.stats import linregress
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')
# import foursq_helper as foursq
# from yelp_helper import *

############################################
########## Plot Style Declaration ##########
# Set the style globally
# Alternatives include bmh, fivethirtyeight, ggplot,
# dark_background, seaborn-deep, etc
# plt.style.use('ggplot')
plt.style.use('seaborn-white')

plt.rcParams['font.family'] = 'times new roman'
# plt.rcParams['font.serif'] = 'Ubuntu'
# plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 15
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

##########################################
########## data set declaration ##########
path_processed = "data/processed/"
path_final = "data/final/"

##################################
########## End of Setup ##########

##### Geographical Change #####
import geopy
from geopy import distance as geopy_distance
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)
print (geopy.distance.distance(coords_1, coords_2).km)

279.35290160430094


In [29]:
X_train = np.load("data/opp_X_train.npy")
X_test = np.load("data/opp_X_test.npy")
y_train = np.load("data/opp_y_train.npy")
y_test = np.load("data/opp_y_test.npy")

In [30]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot
from keras.models import load_model
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from sklearn.metrics import *
from sklearn.metrics import fbeta_score
from keras import backend as K

def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [ ]:
def makeDataD2OneClass(class_num, X_train, y_train, X_test, y_test):
    X_train = X_train[y_train[:]>0]
    y_train = y_train[y_train[:]>0]
    y_train = y_train[:] - 1

    X_test = X_test[y_test[:]>0]
    y_test = y_test[y_test[:]>0]
    y_test = y_test[:] - 1

    X_train_D1 = X_train[y_train[:]<=8]
    X_train_D2 = X_train[y_train[:]>8]

    y_train_D1 = y_train[y_train[:]<=8]
    y_train_D2 = y_train[y_train[:]>8]

    X_test_D1 = X_test[y_test[:]<=8]
    X_test_D2 = X_test[y_test[:]>8]

    y_test_D1 = y_test[y_test[:]<=8]
    y_test_D2 = y_test[y_test[:]>8]
    
    
    return X_train_D1, y_train_D1, X_train_D2, y_train_D2, X_test_D1, y_test_D1, X_test_D2, y_test_D2

In [31]:
X_train = X_train[y_train[:]>0]
y_train = y_train[y_train[:]>0]
y_train = y_train[:] - 1

X_test = X_test[y_test[:]>0]
y_test = y_test[y_test[:]>0]
y_test = y_test[:] - 1

X_train_D1 = X_train[y_train[:]<=8]
X_train_D2 = X_train[y_train[:]>8]

y_train_D1 = y_train[y_train[:]<=8]
y_train_D2 = y_train[y_train[:]>8]

X_test_D1 = X_test[y_test[:]<=8]
X_test_D2 = X_test[y_test[:]>8]

y_test_D1 = y_test[y_test[:]<=8]
y_test_D2 = y_test[y_test[:]>8]

In [44]:
set(y_test)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}

In [61]:
class_l = {9, 10, 11, 12, 13, 14, 15, 16}

In [50]:
y_train[:]>8

array([False, False, False, ...,  True,  True,  True])

In [75]:
a = [True if s in class_l else False for s in y_train]

In [76]:
b = y_train[:]>8

In [69]:
X_train_D2_ = X_train[[True if s in class_l else False for s in y_train]]

In [68]:
X_train_D2 = X_train[y_train[:]>8]

In [74]:
X_train_D2.shape

(7189, 24, 113)

In [73]:
X_train_D2_.shape

(7189, 24, 113)

In [77]:
for v1, v2 in zip(a, b):
    if v1 != v2:
        print("not same")

In [ ]:
X_train_D1 = X_train[y_train[:] is in class_l]

In [91]:
def makeDataD2OneClass(class_num_D2, X_train, y_train, X_test, y_test):

    X_train_D1 = X_train[y_train[:] != class_num_D2]
    X_train_D2 = X_train[y_train[:] == class_num_D2]

    y_train_D1 = y_train[y_train[:] != class_num_D2]
    y_train_D2 = y_train[y_train[:] == class_num_D2]

    X_test_D1 = X_test[y_test[:] != class_num_D2]
    X_test_D2 = X_test[y_test[:] == class_num_D2]

    y_test_D1 = y_test[y_test[:] != class_num_D2]
    y_test_D2 = y_test[y_test[:] == class_num_D2]
    
    return X_train_D1, y_train_D1, X_train_D2, y_train_D2, X_test_D1, y_test_D1, X_test_D2, y_test_D2

# [True if s in class_l else False for s in y_train]
def makeDataD2HalfClasses(class_dict_D1, class_dict_D2, X_train, y_train, X_test, y_test):

    X_train_D1 = X_train[[True if x in class_dict_D1 else False for x in y_train]]
    X_train_D2 = X_train[[True if x in class_dict_D2 else False for x in y_train]]

    y_train_D1 = y_train[[True if x in class_dict_D1 else False for x in y_train]]
    y_train_D2 = y_train[[True if x in class_dict_D2 else False for x in y_train]]

    X_test_D1 = X_test[[True if x in class_dict_D1 else False for x in y_test]]
    X_test_D2 = X_test[[True if x in class_dict_D2 else False for x in y_test]]

    y_test_D1 = y_test[[True if x in class_dict_D1 else False for x in y_test]]
    y_test_D2 = y_test[[True if x in class_dict_D2 else False for x in y_test]]
    
    return X_train_D1, y_train_D1, X_train_D2, y_train_D2, X_test_D1, y_test_D1, X_test_D2, y_test_D2

def makeClassDictD1(class_dict_D2, num_classes):
    class_list = []
    for i in range(num_classes):
        if i not in class_dict_D2:
            class_list.append(i)
    return set(class_list)

In [32]:
def dataPrepare(rand, df_l):
	train_ratio = 0.9
	X_temp, y_res = df_l[0], df_l[1]
	cut = int(X_temp.shape[0] * train_ratio)
    
	#labelencoder_y_1 = LabelEncoder()
	#y_res = labelencoder_y_1.fit_transform(y_res)
    
	train_y = y_res[:cut]
	test_y = y_res[cut:]
	train_X = X_temp[:cut]
	test_X = X_temp[cut:]

	# train_X = StandardScaler().fit_transform(train_X)
	# test_X = StandardScaler().fit_transform(test_X)

	train_X = np.reshape(train_X, (len(train_X),timesteps,-1) )
	test_X = np.reshape(test_X, (len(test_X),timesteps,-1) )

	return train_X, train_y, test_X, test_y

def runKerasModels(data_name, num_stack, hidden_layer_size, df_l):
	auc_l = []
	hist_acc = []
	hist_prec = []
	hist_rec = []
	hist_f1 = []
	path_name = 'data/mlresultsV_sanity/'
	for rand in range(1):
		best_model_name = path_name+data_name+'_'+str(rand)+'_best_model_'+str(timesteps)+'0_'+str(num_stack)+'_'+str(hidden_layer_size)+'.h5'
		train_X, train_y, test_X, test_y = dataPrepare(rand, df_l)
		print(train_X.shape, train_y.shape)
        # X = [input_size, timesteps, data_dim]
        # y = [input_size, num_classes]
		if num_stack == 1:
			# expected input data shape: (batch_size, timesteps, data_dim)
			model = Sequential()
			model.add(LSTM(hidden_layer_size,input_shape=(timesteps, data_dim),init='glorot_normal'))  # return a single vector of dimension 32
			model.add(Dropout(0.2))
			model.add(Dense(num_classes, activation='softmax', init='glorot_normal'))
		elif num_stack == 2:
			model = Sequential()
			model.add(LSTM(hidden_layer_size,return_sequences=True,input_shape=(timesteps, data_dim),init='glorot_normal'))  # returns a sequence of vectors of dimension 32
			model.add(LSTM(hidden_layer_size,init='glorot_normal'))  # return a single vector of dimension 32
			model.add(Dropout(0.2))
			model.add(Dense(num_classes, activation='softmax',init='glorot_normal'))
		elif num_stack == 4:
			model = Sequential()
			model.add(LSTM(hidden_layer_size,return_sequences=True,input_shape=(timesteps, data_dim),init='glorot_normal'))  # returns a sequence of vectors of dimension 32
			model.add(LSTM(hidden_layer_size,return_sequences=True,init='glorot_normal'))  # returns a sequence of vectors of dimension 32
			model.add(LSTM(hidden_layer_size,return_sequences=True,init='glorot_normal'))  # returns a sequence of vectors of dimension 32
			model.add(LSTM(hidden_layer_size,init='glorot_normal'))  # return a single vector of dimension 32
			model.add(Dropout(0.2))
			model.add(Dense(num_classes, activation='softmax',init='glorot_normal'))
		else:
			print("specify the number of stacks")
		# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# 		model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=[fbeta])
		model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['sparse_categorical_accuracy'])
		# simple early stopping
		es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=60)
		mc = ModelCheckpoint(best_model_name, monitor='val_acc', mode='max', verbose=1, save_best_only=True)
		history = model.fit(train_X, train_y, validation_data=(test_X, test_y),epochs=10,batch_size=64,verbose=0, callbacks=[es, mc])
		saved_model = load_model(best_model_name)
		test_pred = np.argmax(saved_model.predict(test_X), axis=1)
		acc = accuracy_score(test_y, test_pred)
		prfs = precision_recall_fscore_support(test_y, test_pred)
		print("Epoch ", t, "acc / prec / recall / f1 / supp : ", acc," ", prfs)
		hist_acc.append(acc)
		hist_prec.append(prfs[0])
		hist_rec.append(prfs[1])
		hist_f1.append(prfs[2])        
# 		auc_score = roc_auc_score(test_y, saved_model.predict(test_X, verbose=0))
# 		auc_l.append(auc_score)
# 		print('AUC score: %f' %(auc_score))
	return hist_acc, hist_prec, hist_rec, hist_f1

In [33]:
data_dim = 113 # (# features)
num_classes = 17

# params
timesteps = 24
num_stack = 1
hidden_layer_size = 128

# train_X, train_y, test_X, test_y = dataPrepare(1, [X_train_D1, y_train_D1])

model = Sequential()
model.add(LSTM(hidden_layer_size,input_shape=(timesteps, data_dim),init='glorot_normal'))  # return a single vector of dimension 32
# model.add(LSTM(hidden_layer_size,return_sequences=True,input_shape=(timesteps, data_dim),init='glorot_normal'))  # returns a sequence of vectors of dimension 32
# model.add(LSTM(hidden_layer_size,init='glorot_normal'))  # return a single vector of dimension 32
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax',init='glorot_normal'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['sparse_categorical_accuracy'])




In [92]:
X_test.shape

(1657, 24, 113)

In [95]:
X_train_linear = X_train.reshape(-1, timesteps*data_dim)
X_test_linear = X_test.reshape(-1, timesteps*data_dim)

X_train_D1_linear = X_train_D1.reshape(-1, timesteps*data_dim)
X_test_D1_linear = X_test_D1.reshape(-1, timesteps*data_dim)

X_train_D2_linear = X_train_D2.reshape(-1, timesteps*data_dim)
X_test_D2_linear = X_test_D2.reshape(-1, timesteps*data_dim)

In [94]:
X_test_linear.shape

(1657, 2712)

In [98]:
model = Sequential()
model.add(Dense(1024, activation='tanh', input_shape=(data_dim*timesteps,)))
# model.add(Dropout(0.2))
model.add(Dense(512, activation='tanh'))
# model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax',init='glorot_normal'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['sparse_categorical_accuracy'])

In [99]:
##### MLPs Offilne baseline
model.fit(X_train_linear, y_train, validation_data=(X_test_linear, y_test),
                    epochs=100,batch_size=64)

Train on 14147 samples, validate on 1657 samples
Epoch 1/100
14147/14147 [==============================] - 17s 1ms/step - loss: 3.1398 - sparse_categorical_accuracy: 0.2601 - val_loss: 3.4401 - val_sparse_categorical_accuracy: 0.1907
Epoch 2/100
14147/14147 [==============================] - 10s 728us/step - loss: 2.5858 - sparse_categorical_accuracy: 0.3597 - val_loss: 3.3623 - val_sparse_categorical_accuracy: 0.3241
Epoch 3/100
14147/14147 [==============================] - 9s 634us/step - loss: 2.3801 - sparse_categorical_accuracy: 0.4170 - val_loss: 2.2367 - val_sparse_categorical_accuracy: 0.2945
Epoch 4/100
14147/14147 [==============================] - 9s 651us/step - loss: 1.4669 - sparse_categorical_accuracy: 0.4672 - val_loss: 1.8043 - val_sparse_categorical_accuracy: 0.3651
Epoch 5/100
14147/14147 [==============================] - 11s 775us/step - loss: 1.3273 - sparse_categorical_accuracy: 0.4976 - val_loss: 1.6742 - val_sparse_categorical_accuracy: 0.4448
Epoch 6/100
141

14147/14147 [==============================] - 8s 563us/step - loss: 0.8811 - sparse_categorical_accuracy: 0.6117 - val_loss: 1.5456 - val_sparse_categorical_accuracy: 0.4412
Epoch 45/100
14147/14147 [==============================] - 8s 580us/step - loss: 0.9078 - sparse_categorical_accuracy: 0.6096 - val_loss: 1.2954 - val_sparse_categorical_accuracy: 0.4888
Epoch 46/100
14147/14147 [==============================] - 8s 587us/step - loss: 0.8946 - sparse_categorical_accuracy: 0.6071 - val_loss: 1.3903 - val_sparse_categorical_accuracy: 0.4605
Epoch 47/100
14147/14147 [==============================] - 8s 582us/step - loss: 0.9311 - sparse_categorical_accuracy: 0.6020 - val_loss: 1.5253 - val_sparse_categorical_accuracy: 0.5027
Epoch 48/100
14147/14147 [==============================] - 8s 581us/step - loss: 0.8694 - sparse_categorical_accuracy: 0.6186 - val_loss: 1.4715 - val_sparse_categorical_accuracy: 0.4478
Epoch 49/100
14147/14147 [==============================] - 8s 577us/step

14147/14147 [==============================] - 8s 593us/step - loss: 1.4432 - sparse_categorical_accuracy: 0.4575 - val_loss: 1.9174 - val_sparse_categorical_accuracy: 0.3211
Epoch 88/100
14147/14147 [==============================] - 9s 651us/step - loss: 1.4230 - sparse_categorical_accuracy: 0.4598 - val_loss: 2.0044 - val_sparse_categorical_accuracy: 0.3259
Epoch 89/100
14147/14147 [==============================] - 9s 605us/step - loss: 1.4074 - sparse_categorical_accuracy: 0.4676 - val_loss: 1.9349 - val_sparse_categorical_accuracy: 0.3337
Epoch 90/100
14147/14147 [==============================] - 9s 613us/step - loss: 1.4020 - sparse_categorical_accuracy: 0.4721 - val_loss: 1.8418 - val_sparse_categorical_accuracy: 0.4056
Epoch 91/100
14147/14147 [==============================] - 9s 641us/step - loss: 1.3919 - sparse_categorical_accuracy: 0.4773 - val_loss: 1.7729 - val_sparse_categorical_accuracy: 0.3730
Epoch 92/100
14147/14147 [==============================] - 9s 666us/step

In [114]:
model = Sequential()
model.add(Dense(1024, activation='tanh', input_shape=(data_dim*timesteps,)))
# model.add(Dropout(0.2))
model.add(Dense(512, activation='tanh'))
# model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax',init='glorot_normal'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['sparse_categorical_accuracy'])

In [116]:
##### MLPs Offilne baseline
model.fit(X_train_D1_linear, y_train_D1, validation_data=(X_test_D1_linear, y_test_D1),
                    epochs=100,batch_size=64)

Train on 10901 samples, validate on 1340 samples
Epoch 1/100
10901/10901 [==============================] - 12s 1ms/step - loss: 2.5983 - sparse_categorical_accuracy: 0.1270 - val_loss: 2.2987 - val_sparse_categorical_accuracy: 0.1000
Epoch 2/100
10901/10901 [==============================] - 10s 903us/step - loss: 2.1562 - sparse_categorical_accuracy: 0.2048 - val_loss: 2.0009 - val_sparse_categorical_accuracy: 0.2993
Epoch 3/100
10901/10901 [==============================] - 8s 747us/step - loss: 1.9024 - sparse_categorical_accuracy: 0.2872 - val_loss: 1.9006 - val_sparse_categorical_accuracy: 0.3425
Epoch 4/100
10901/10901 [==============================] - 8s 760us/step - loss: 1.7003 - sparse_categorical_accuracy: 0.3308 - val_loss: 1.7281 - val_sparse_categorical_accuracy: 0.3493
Epoch 5/100
10901/10901 [==============================] - 8s 778us/step - loss: 1.6113 - sparse_categorical_accuracy: 0.3474 - val_loss: 1.6885 - val_sparse_categorical_accuracy: 0.3537
Epoch 6/100
1090

KeyboardInterrupt: 

In [108]:
X_test_linear[0].shape

(2712,)

In [112]:
Counter(test_pred)

Counter({0: 97, 3: 281, 5: 665, 12: 246, 14: 83, 15: 214, 16: 71})

In [113]:
Counter(y_test)

Counter({0: 58,
         1: 95,
         2: 60,
         3: 83,
         4: 228,
         5: 160,
         6: 100,
         7: 77,
         8: 39,
         9: 42,
         10: 40,
         11: 26,
         12: 67,
         13: 61,
         14: 99,
         15: 317,
         16: 105})

In [118]:
Counter(test_pred)

Counter({0: 52,
         1: 40,
         2: 332,
         3: 166,
         4: 104,
         5: 205,
         6: 3,
         7: 374,
         8: 43,
         11: 27,
         13: 120,
         14: 147,
         16: 44})

In [117]:
test_pred = np.argmax(model.predict(X_test_linear), axis=1)
accuracy_score(y_test, test_pred)

0.3035606517803259

In [90]:
##### LSTMs offline baseline
model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=100,batch_size=64)

Train on 14147 samples, validate on 1657 samples
Epoch 1/100
14147/14147 [==============================] - 11s 766us/step - loss: 1.4098 - sparse_categorical_accuracy: 0.5345 - val_loss: 1.3480 - val_sparse_categorical_accuracy: 0.5963
Epoch 2/100
14147/14147 [==============================] - 10s 722us/step - loss: 0.7282 - sparse_categorical_accuracy: 0.7192 - val_loss: 1.1549 - val_sparse_categorical_accuracy: 0.6150
Epoch 3/100
14147/14147 [==============================] - 11s 781us/step - loss: 0.6107 - sparse_categorical_accuracy: 0.7679 - val_loss: 0.9708 - val_sparse_categorical_accuracy: 0.6566
Epoch 4/100
14147/14147 [==============================] - 11s 764us/step - loss: 0.5572 - sparse_categorical_accuracy: 0.7861 - val_loss: 0.9715 - val_sparse_categorical_accuracy: 0.6729
Epoch 5/100
14147/14147 [==============================] - 11s 775us/step - loss: 0.5101 - sparse_categorical_accuracy: 0.8029 - val_loss: 1.2193 - val_sparse_categorical_accuracy: 0.5969
Epoch 6/100

14147/14147 [==============================] - 16s 1ms/step - loss: 0.2439 - sparse_categorical_accuracy: 0.9076 - val_loss: 0.7797 - val_sparse_categorical_accuracy: 0.7719
Epoch 45/100
14147/14147 [==============================] - 12s 846us/step - loss: 0.2388 - sparse_categorical_accuracy: 0.9105 - val_loss: 0.8734 - val_sparse_categorical_accuracy: 0.7465
Epoch 46/100
14147/14147 [==============================] - 12s 836us/step - loss: 0.2327 - sparse_categorical_accuracy: 0.9108 - val_loss: 0.8742 - val_sparse_categorical_accuracy: 0.7532
Epoch 47/100
14147/14147 [==============================] - 12s 840us/step - loss: 0.2627 - sparse_categorical_accuracy: 0.9001 - val_loss: 0.7694 - val_sparse_categorical_accuracy: 0.7526
Epoch 48/100
14147/14147 [==============================] - 12s 829us/step - loss: 0.2066 - sparse_categorical_accuracy: 0.9229 - val_loss: 0.9745 - val_sparse_categorical_accuracy: 0.7417
Epoch 49/100
14147/14147 [==============================] - 12s 856us/

14147/14147 [==============================] - 8s 553us/step - loss: 0.4123 - sparse_categorical_accuracy: 0.8474 - val_loss: 0.8842 - val_sparse_categorical_accuracy: 0.7477
Epoch 88/100
14147/14147 [==============================] - 7s 512us/step - loss: 0.3034 - sparse_categorical_accuracy: 0.8859 - val_loss: 1.0936 - val_sparse_categorical_accuracy: 0.6982
Epoch 89/100
14147/14147 [==============================] - 9s 640us/step - loss: 0.3633 - sparse_categorical_accuracy: 0.8694 - val_loss: 0.7590 - val_sparse_categorical_accuracy: 0.7749
Epoch 90/100
14147/14147 [==============================] - 9s 639us/step - loss: 0.2162 - sparse_categorical_accuracy: 0.9208 - val_loss: 0.8109 - val_sparse_categorical_accuracy: 0.7713
Epoch 91/100
14147/14147 [==============================] - 7s 529us/step - loss: 0.1836 - sparse_categorical_accuracy: 0.9331 - val_loss: 0.8864 - val_sparse_categorical_accuracy: 0.7598
Epoch 92/100
14147/14147 [==============================] - 7s 466us/step

In [80]:
def vanilaLSTMs():
    model = Sequential()
    model.add(LSTM(hidden_layer_size,input_shape=(timesteps, data_dim),init='glorot_normal'))  # return a single vector of dimension 32
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax',init='glorot_normal'))
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['sparse_categorical_accuracy'])
    return model

In [81]:
data_dim = 113 # (# features)
num_classes = 17

# params
timesteps = 24
num_stack = 1
hidden_layer_size = 128

In [84]:
Counter(y_train)

Counter({0: 864,
         1: 887,
         2: 806,
         3: 846,
         4: 921,
         5: 850,
         6: 666,
         7: 628,
         8: 490,
         9: 413,
         10: 457,
         11: 416,
         12: 566,
         13: 564,
         14: 904,
         15: 3246,
         16: 623})

In [ ]:
print("==== Baseline model performance ====")
model = vanilaLSTMs()
model.fit(X_train, y_train, validation_data=(X_test, y_test),
                        epochs=100,batch_size=64)

In [85]:
class_num_l = [0,4,8,14,15]

class_num_D2=0
print("==== Adding One More Class ====")
print("class num D2: %d" %(class_num_D2))
X_train_D1, y_train_D1, X_train_D2, y_train_D2, X_test_D1, y_test_D1, X_test_D2, y_test_D2 = makeDataD2OneClass(class_num_D2, X_train, y_train, X_test, y_test)
model = vanilaLSTMs()
model.fit(X_train_D1, y_train_D1, validation_data=(X_test_D1, y_test_D1),
                    epochs=30,batch_size=64)
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=1,batch_size=64)
print("Performances after Epoch: 1")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=9,batch_size=64)
print("Performances after Epoch: 10")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=20,batch_size=64)
print("Performances after Epoch: 30")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))

==== Adding One More Class ====
class num D2: 0
Train on 13283 samples, validate on 1599 samples
Epoch 1/30
13283/13283 [==============================] - 10s 723us/step - loss: 1.7548 - sparse_categorical_accuracy: 0.3969 - val_loss: 1.6526 - val_sparse_categorical_accuracy: 0.4378
Epoch 2/30
13283/13283 [==============================] - 8s 567us/step - loss: 1.2415 - sparse_categorical_accuracy: 0.5375 - val_loss: 1.4795 - val_sparse_categorical_accuracy: 0.4415
Epoch 3/30
13283/13283 [==============================] - 8s 593us/step - loss: 1.0091 - sparse_categorical_accuracy: 0.6104 - val_loss: 1.2266 - val_sparse_categorical_accuracy: 0.5629
Epoch 4/30
13283/13283 [==============================] - 8s 607us/step - loss: 0.8854 - sparse_categorical_accuracy: 0.6560 - val_loss: 1.2101 - val_sparse_categorical_accuracy: 0.5560
Epoch 5/30
13283/13283 [==============================] - 7s 520us/step - loss: 0.8240 - sparse_categorical_accuracy: 0.6740 - val_loss: 1.0588 - val_sparse_c

864/864 [==============================] - 1s 761us/step - loss: 3.1004e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 12.2959 - val_sparse_categorical_accuracy: 0.0350
Epoch 4/20
864/864 [==============================] - 1s 732us/step - loss: 2.5711e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 12.3481 - val_sparse_categorical_accuracy: 0.0350
Epoch 5/20
864/864 [==============================] - 1s 772us/step - loss: 2.4931e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 12.3971 - val_sparse_categorical_accuracy: 0.0350
Epoch 6/20
864/864 [==============================] - 1s 1ms/step - loss: 2.3913e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 12.4453 - val_sparse_categorical_accuracy: 0.0350
Epoch 7/20
864/864 [==============================] - 1s 982us/step - loss: 1.9979e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 12.4883 - val_sparse_categorical_accuracy: 0.0350
Epoch 8/20
864/864 [==============================] - 1s 2ms/step - loss: 2.02

In [86]:
class_num_D2=4
print("==== Adding One More Class ====")
print("class num D2: %d" %(class_num_D2))
X_train_D1, y_train_D1, X_train_D2, y_train_D2, X_test_D1, y_test_D1, X_test_D2, y_test_D2 = makeDataD2OneClass(class_num_D2, X_train, y_train, X_test, y_test)
model = vanilaLSTMs()
model.fit(X_train_D1, y_train_D1, validation_data=(X_test_D1, y_test_D1),
                    epochs=30,batch_size=64)
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=1,batch_size=64)
print("Performances after Epoch: 1")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=9,batch_size=64)
print("Performances after Epoch: 10")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=20,batch_size=64)
print("Performances after Epoch: 30")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))

==== Adding One More Class ====
class num D2: 4
Train on 13226 samples, validate on 1429 samples
Epoch 1/30
13226/13226 [==============================] - 8s 627us/step - loss: 1.7727 - sparse_categorical_accuracy: 0.3808 - val_loss: 1.7145 - val_sparse_categorical_accuracy: 0.4059
Epoch 2/30
13226/13226 [==============================] - 6s 478us/step - loss: 1.2566 - sparse_categorical_accuracy: 0.5197 - val_loss: 1.5621 - val_sparse_categorical_accuracy: 0.4808
Epoch 3/30
13226/13226 [==============================] - 6s 482us/step - loss: 1.0273 - sparse_categorical_accuracy: 0.5962 - val_loss: 1.3871 - val_sparse_categorical_accuracy: 0.5122
Epoch 4/30
13226/13226 [==============================] - 6s 486us/step - loss: 0.9070 - sparse_categorical_accuracy: 0.6412 - val_loss: 1.2353 - val_sparse_categorical_accuracy: 0.5563
Epoch 5/30
13226/13226 [==============================] - 7s 527us/step - loss: 0.8159 - sparse_categorical_accuracy: 0.6706 - val_loss: 1.1321 - val_sparse_ca

921/921 [==============================] - 1s 730us/step - loss: 6.7769e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 10.1094 - val_sparse_categorical_accuracy: 0.1376
Epoch 4/20
921/921 [==============================] - 1s 786us/step - loss: 6.3183e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 10.1591 - val_sparse_categorical_accuracy: 0.1376
Epoch 5/20
921/921 [==============================] - 1s 745us/step - loss: 5.7292e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 10.2032 - val_sparse_categorical_accuracy: 0.1376
Epoch 6/20
921/921 [==============================] - 1s 738us/step - loss: 5.4355e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 10.2437 - val_sparse_categorical_accuracy: 0.1376
Epoch 7/20
921/921 [==============================] - 1s 672us/step - loss: 5.0584e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 10.2797 - val_sparse_categorical_accuracy: 0.1376
Epoch 8/20
921/921 [==============================] - 1s 741us/step - loss: 

In [87]:
class_num_D2=8
print("==== Adding One More Class ====")
print("class num D2: %d" %(class_num_D2))
X_train_D1, y_train_D1, X_train_D2, y_train_D2, X_test_D1, y_test_D1, X_test_D2, y_test_D2 = makeDataD2OneClass(class_num_D2, X_train, y_train, X_test, y_test)
model = vanilaLSTMs()
model.fit(X_train_D1, y_train_D1, validation_data=(X_test_D1, y_test_D1),
                    epochs=30,batch_size=64)
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=1,batch_size=64)
print("Performances after Epoch: 1")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=9,batch_size=64)
print("Performances after Epoch: 10")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=20,batch_size=64)
print("Performances after Epoch: 30")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))

==== Adding One More Class ====
class num D2: 8
Train on 13657 samples, validate on 1618 samples
Epoch 1/30
13657/13657 [==============================] - 9s 646us/step - loss: 1.6986 - sparse_categorical_accuracy: 0.3996 - val_loss: 1.6807 - val_sparse_categorical_accuracy: 0.4178
Epoch 2/30
13657/13657 [==============================] - 7s 500us/step - loss: 1.1956 - sparse_categorical_accuracy: 0.5304 - val_loss: 1.5508 - val_sparse_categorical_accuracy: 0.4821
Epoch 3/30
13657/13657 [==============================] - 7s 484us/step - loss: 1.0070 - sparse_categorical_accuracy: 0.5961 - val_loss: 1.4518 - val_sparse_categorical_accuracy: 0.4623
Epoch 4/30
13657/13657 [==============================] - 7s 495us/step - loss: 0.9078 - sparse_categorical_accuracy: 0.6339 - val_loss: 1.2189 - val_sparse_categorical_accuracy: 0.5389
Epoch 5/30
13657/13657 [==============================] - 7s 546us/step - loss: 0.8275 - sparse_categorical_accuracy: 0.6616 - val_loss: 1.2853 - val_sparse_ca

490/490 [==============================] - 0s 908us/step - loss: 7.3538e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 11.0971 - val_sparse_categorical_accuracy: 0.0235
Epoch 4/20
490/490 [==============================] - 0s 858us/step - loss: 7.6480e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 11.1415 - val_sparse_categorical_accuracy: 0.0235
Epoch 5/20
490/490 [==============================] - 0s 930us/step - loss: 6.2557e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 11.1832 - val_sparse_categorical_accuracy: 0.0235
Epoch 6/20
490/490 [==============================] - 0s 904us/step - loss: 6.9739e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 11.2245 - val_sparse_categorical_accuracy: 0.0235
Epoch 7/20
490/490 [==============================] - 0s 903us/step - loss: 6.8222e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 11.2642 - val_sparse_categorical_accuracy: 0.0235
Epoch 8/20
490/490 [==============================] - 0s 876us/step - loss: 

In [88]:
class_num_D2=14
print("==== Adding One More Class ====")
print("class num D2: %d" %(class_num_D2))
X_train_D1, y_train_D1, X_train_D2, y_train_D2, X_test_D1, y_test_D1, X_test_D2, y_test_D2 = makeDataD2OneClass(class_num_D2, X_train, y_train, X_test, y_test)
model = vanilaLSTMs()
model.fit(X_train_D1, y_train_D1, validation_data=(X_test_D1, y_test_D1),
                    epochs=30,batch_size=64)
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=1,batch_size=64)
print("Performances after Epoch: 1")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=9,batch_size=64)
print("Performances after Epoch: 10")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=20,batch_size=64)
print("Performances after Epoch: 30")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))

==== Adding One More Class ====
class num D2: 14
Train on 13243 samples, validate on 1558 samples
Epoch 1/30
13243/13243 [==============================] - 10s 720us/step - loss: 1.7785 - sparse_categorical_accuracy: 0.3828 - val_loss: 1.8131 - val_sparse_categorical_accuracy: 0.3780
Epoch 2/30
13243/13243 [==============================] - 7s 521us/step - loss: 1.3089 - sparse_categorical_accuracy: 0.4981 - val_loss: 1.5351 - val_sparse_categorical_accuracy: 0.4551
Epoch 3/30
13243/13243 [==============================] - 7s 530us/step - loss: 1.1219 - sparse_categorical_accuracy: 0.5589 - val_loss: 1.4606 - val_sparse_categorical_accuracy: 0.4775
Epoch 4/30
13243/13243 [==============================] - 8s 616us/step - loss: 1.0026 - sparse_categorical_accuracy: 0.5977 - val_loss: 1.1855 - val_sparse_categorical_accuracy: 0.5385
Epoch 5/30
13243/13243 [==============================] - 14s 1ms/step - loss: 0.9269 - sparse_categorical_accuracy: 0.6181 - val_loss: 1.2497 - val_sparse_c

904/904 [==============================] - 2s 2ms/step - loss: 2.3719e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 11.6586 - val_sparse_categorical_accuracy: 0.0597
Epoch 4/20
904/904 [==============================] - 2s 2ms/step - loss: 2.4051e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 11.6867 - val_sparse_categorical_accuracy: 0.0597
Epoch 5/20
904/904 [==============================] - 2s 2ms/step - loss: 1.9773e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 11.7144 - val_sparse_categorical_accuracy: 0.0597
Epoch 6/20
904/904 [==============================] - 2s 2ms/step - loss: 1.8719e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 11.7400 - val_sparse_categorical_accuracy: 0.0597
Epoch 7/20
904/904 [==============================] - 2s 2ms/step - loss: 1.8330e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 11.7647 - val_sparse_categorical_accuracy: 0.0597
Epoch 8/20
904/904 [==============================] - 1s 2ms/step - loss: 1.9135e-04 -

In [89]:
class_num_D2=15
print("==== Adding One More Class ====")
print("class num D2: %d" %(class_num_D2))
X_train_D1, y_train_D1, X_train_D2, y_train_D2, X_test_D1, y_test_D1, X_test_D2, y_test_D2 = makeDataD2OneClass(class_num_D2, X_train, y_train, X_test, y_test)
model = vanilaLSTMs()
model.fit(X_train_D1, y_train_D1, validation_data=(X_test_D1, y_test_D1),
                    epochs=30,batch_size=64)
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=1,batch_size=64)
print("Performances after Epoch: 1")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=9,batch_size=64)
print("Performances after Epoch: 10")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=20,batch_size=64)
print("Performances after Epoch: 30")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))

==== Adding One More Class ====
class num D2: 15
Train on 10901 samples, validate on 1340 samples
Epoch 1/30
10901/10901 [==============================] - 14s 1ms/step - loss: 2.0221 - sparse_categorical_accuracy: 0.2545 - val_loss: 1.6792 - val_sparse_categorical_accuracy: 0.3851
Epoch 2/30
10901/10901 [==============================] - 11s 1ms/step - loss: 1.5555 - sparse_categorical_accuracy: 0.3919 - val_loss: 1.4610 - val_sparse_categorical_accuracy: 0.3963
Epoch 3/30
10901/10901 [==============================] - 9s 838us/step - loss: 1.3181 - sparse_categorical_accuracy: 0.4665 - val_loss: 1.6257 - val_sparse_categorical_accuracy: 0.3590
Epoch 4/30
10901/10901 [==============================] - 10s 917us/step - loss: 1.1543 - sparse_categorical_accuracy: 0.5303 - val_loss: 1.3017 - val_sparse_categorical_accuracy: 0.4813
Epoch 5/30
10901/10901 [==============================] - 11s 968us/step - loss: 1.0803 - sparse_categorical_accuracy: 0.5497 - val_loss: 1.2055 - val_sparse_c

Epoch 3/20
3246/3246 [==============================] - 3s 964us/step - loss: 7.2241e-05 - sparse_categorical_accuracy: 1.0000 - val_loss: 10.2363 - val_sparse_categorical_accuracy: 0.1913
Epoch 4/20
3246/3246 [==============================] - 3s 960us/step - loss: 7.3508e-05 - sparse_categorical_accuracy: 1.0000 - val_loss: 10.2826 - val_sparse_categorical_accuracy: 0.1913
Epoch 5/20
3246/3246 [==============================] - 3s 881us/step - loss: 7.1636e-05 - sparse_categorical_accuracy: 1.0000 - val_loss: 10.3309 - val_sparse_categorical_accuracy: 0.1913
Epoch 6/20
3246/3246 [==============================] - 3s 958us/step - loss: 6.3827e-05 - sparse_categorical_accuracy: 1.0000 - val_loss: 10.3778 - val_sparse_categorical_accuracy: 0.1913
Epoch 7/20
3246/3246 [==============================] - 3s 982us/step - loss: 5.6742e-05 - sparse_categorical_accuracy: 1.0000 - val_loss: 10.4184 - val_sparse_categorical_accuracy: 0.1913
Epoch 8/20
3246/3246 [==============================] -

In [36]:
model.fit(X_train_D1, y_train_D1, validation_data=(X_test_D1, y_test_D1),
                    epochs=30,batch_size=64)


Train on 6958 samples, validate on 900 samples
Epoch 1/30
6958/6958 [==============================] - 6s 811us/step - loss: 1.7175 - sparse_categorical_accuracy: 0.2823 - val_loss: 1.5340 - val_sparse_categorical_accuracy: 0.3822
Epoch 2/30
6958/6958 [==============================] - 4s 557us/step - loss: 1.2685 - sparse_categorical_accuracy: 0.4003 - val_loss: 1.4047 - val_sparse_categorical_accuracy: 0.4011
Epoch 3/30
6958/6958 [==============================] - 4s 524us/step - loss: 1.2081 - sparse_categorical_accuracy: 0.4395 - val_loss: 1.2176 - val_sparse_categorical_accuracy: 0.4800
Epoch 4/30
6958/6958 [==============================] - 4s 525us/step - loss: 1.1081 - sparse_categorical_accuracy: 0.4931 - val_loss: 1.2115 - val_sparse_categorical_accuracy: 0.4222
Epoch 5/30
6958/6958 [==============================] - 4s 559us/step - loss: 1.0406 - sparse_categorical_accuracy: 0.5277 - val_loss: 1.0925 - val_sparse_categorical_accuracy: 0.5078
Epoch 6/30
6958/6958 [===========

In [37]:
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=30,batch_size=64)

Train on 7189 samples, validate on 1657 samples
Epoch 1/30
7189/7189 [==============================] - 4s 586us/step - loss: 1.2322 - sparse_categorical_accuracy: 0.5958 - val_loss: 4.7684 - val_sparse_categorical_accuracy: 0.2824
Epoch 2/30
7189/7189 [==============================] - 4s 551us/step - loss: 0.6821 - sparse_categorical_accuracy: 0.7140 - val_loss: 4.9061 - val_sparse_categorical_accuracy: 0.2993
Epoch 3/30
7189/7189 [==============================] - 4s 549us/step - loss: 0.6060 - sparse_categorical_accuracy: 0.7499 - val_loss: 4.3866 - val_sparse_categorical_accuracy: 0.3096
Epoch 4/30
7189/7189 [==============================] - 4s 537us/step - loss: 0.5111 - sparse_categorical_accuracy: 0.7841 - val_loss: 4.6187 - val_sparse_categorical_accuracy: 0.3368
Epoch 5/30
7189/7189 [==============================] - 4s 582us/step - loss: 0.4907 - sparse_categorical_accuracy: 0.7950 - val_loss: 4.6365 - val_sparse_categorical_accuracy: 0.3392
Epoch 6/30
7189/7189 [==========

In [40]:
test_pred = np.argmax(model.predict(X_test_D1), axis=1)
accuracy_score(y_test_D1, test_pred)

0.0

In [41]:
test_pred = np.argmax(model.predict(X_test_D2), axis=1)
accuracy_score(y_test_D2, test_pred)

0.8256274768824307

In [42]:
test_pred = np.argmax(model.predict(X_test), axis=1)
accuracy_score(y_test, test_pred)

0.3771876885938443

In [ ]:
data_dim = 113 # (# features)
num_classes = 17

# params
timesteps_l = [24]
num_stack_l = [2]
hidden_layer_size_l = [64]
for timesteps in timesteps_l:
    for num_stack in num_stack_l:
        for hidden_layer_size in hidden_layer_size_l:
            # auc_l = runKerasModels('yelp',num_stack, hidden_layer_size, [X_test, y_test])
            hist_l = runKerasModels('D1',num_stack, hidden_layer_size, [X_train, y_train])
            print("final result: # stacks: %d, hidden_layer_size: %d, mean acc: %f, prec: %f, rec: %f, f1: %f" %(num_stack,hidden_layer_size,np.mean(hist_l[0]),np.mean(hist_l[1]),np.mean(hist_l[2]),np.mean(hist_l[3])))
#             print(hist_l)

In [43]:
[3] + [2]

[3, 2]